In [1]:
#Decoding scripts/ de-compressing scripts

import os
import numpy as np
import rans
from PIL import Image
import util
from torch_vae import tvae_utils
from torch_vae.tvae_beta_binomial import BetaBinomialVAE
import torch
import time

# Parameters
prior_precision = 8
obs_precision = 14
q_precision = 14
latent_dim = 50
latent_shape = (1, latent_dim)

# Load the VAE model
model = BetaBinomialVAE(hidden_dim=200, latent_dim=latent_dim)
model.load_state_dict(
    torch.load('saved_params/torch_vae_beta_binomial_params',
               map_location=lambda storage, location: storage))
model.eval()

rec_net = tvae_utils.torch_fun_to_numpy_fun(model.encode)
gen_net = tvae_utils.torch_fun_to_numpy_fun(model.decode)

obs_append = tvae_utils.beta_binomial_obs_append(255, obs_precision)
obs_pop = tvae_utils.beta_binomial_obs_pop(255, obs_precision)

vae_append = util.vae_append(latent_shape, gen_net, rec_net, obs_append,
                             prior_precision, q_precision)
vae_pop = util.vae_pop(latent_shape, gen_net, rec_net, obs_pop,
                       prior_precision, q_precision)

# Load the compressed message
compressed_lengths = np.loadtxt('compressed_lengths_cts').tolist()
compressed_message = np.fromfile('compressed_message.bin', dtype=np.uint32)


# Decode the images
state = rans.unflatten(compressed_message)
decode_start_time = time.time()
decoded_images = []

for n in range(len(compressed_lengths)):
    state, image_ = vae_pop(state)
    
    # Decompressed image
    image_array = image_.numpy().reshape(28, 28)  # Convert to NumPy and reshape for MNIST dimensions (28x28)
    image_save_path = f'decompressed_image/decompressed_image_{n}.png'
    Image.fromarray(image_array.astype(np.uint8)).save(image_save_path)
    
    decoded_images.append(image_array)
    print(f'Decoded image {n}')

print('\nAll decoded in {:.2f}s'.format(time.time() - decode_start_time))

# Save the recovered bits
recovered_bits = rans.flatten(state)
np.save('recovered_bits.npy', recovered_bits)


/tmp/ipykernel_45356/4203628668.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load('saved_params/torch_vae_beta_binomial_params',


FileNotFoundError: [Errno 2] No such file or directory: '/home/pradeep/Downloads/courses/DIP/project/submission_codes/result_2/decompressed_image_0.png'